In [4]:
!pip install pydeck
!pip install streamlit

You should consider upgrading via the 'c:\users\choon\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\choon\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
import numpy as np
import pandas as pd
import time
import streamlit as st
import requests
import pydeck as pdk
import json

In [6]:
weather_data = requests.get("https://api.data.gov.sg/v1/environment/pm25").json()

In [7]:
weather_data

{'region_metadata': [{'name': 'west',
   'label_location': {'latitude': 1.35735, 'longitude': 103.7}},
  {'name': 'east',
   'label_location': {'latitude': 1.35735, 'longitude': 103.94}},
  {'name': 'central',
   'label_location': {'latitude': 1.35735, 'longitude': 103.82}},
  {'name': 'south',
   'label_location': {'latitude': 1.29587, 'longitude': 103.82}},
  {'name': 'north',
   'label_location': {'latitude': 1.41803, 'longitude': 103.82}}],
 'items': [{'timestamp': '2021-06-26T14:00:00+08:00',
   'update_timestamp': '2021-06-26T14:08:53+08:00',
   'readings': {'pm25_one_hourly': {'west': 5,
     'east': 7,
     'central': 16,
     'south': 22,
     'north': 9}}}],
 'api_info': {'status': 'healthy'}}

In [8]:
region_info = weather_data["region_metadata"]
region_info

[{'name': 'west', 'label_location': {'latitude': 1.35735, 'longitude': 103.7}},
 {'name': 'east',
  'label_location': {'latitude': 1.35735, 'longitude': 103.94}},
 {'name': 'central',
  'label_location': {'latitude': 1.35735, 'longitude': 103.82}},
 {'name': 'south',
  'label_location': {'latitude': 1.29587, 'longitude': 103.82}},
 {'name': 'north',
  'label_location': {'latitude': 1.41803, 'longitude': 103.82}}]

In [9]:
region_info = pd.DataFrame.from_dict(weather_data["region_metadata"])
region_info

,name,label_location
0,west,"{'latitude': 1.35735, 'longitude': 103.7}"
1,east,"{'latitude': 1.35735, 'longitude': 103.94}"
2,central,"{'latitude': 1.35735, 'longitude': 103.82}"
3,south,"{'latitude': 1.29587, 'longitude': 103.82}"
4,north,"{'latitude': 1.41803, 'longitude': 103.82}"


In [10]:
region_info['label_location'].apply(pd.Series)
region_info = pd.concat([region_info.drop(['label_location'],axis=1),region_info['label_location'].apply(pd.Series)],axis=1)

In [11]:
mdpt_lon = region_info['longitude'].mean()
mdpt_lat = region_info['latitude'].mean()

In [12]:
round(mdpt_lon,2)

103.82

In [13]:
pm_readings = weather_data['items']
pm_readings = pm_readings[0]
# convert the readings to 

In [14]:
#processing the pm_readings 
pm_info = pd.DataFrame.from_dict(weather_data['items'][0])
pm_info['readings'].apply(pd.Series)

,west,east,central,south,north
pm25_one_hourly,5,7,16,22,9


In [15]:
pm_info.drop(['readings'],axis=1)

,timestamp,update_timestamp
pm25_one_hourly,2021-06-26T14:00:00+08:00,2021-06-26T14:08:53+08:00


In [16]:
pm_info = pd.concat([pm_info.drop(['readings'],axis=1),pm_info['readings'].apply(pd.Series)],axis=1)

In [17]:
pm_info

,timestamp,update_timestamp,west,east,central,south,north
pm25_one_hourly,2021-06-26T14:00:00+08:00,2021-06-26T14:08:53+08:00,5,7,16,22,9


In [18]:
pm = pm_info[['west','east','central','south','north']].T.reset_index()
pm = pm.rename(columns={"index":"region"})

In [19]:
df = region_info.merge(pm,left_on="name",right_on="region")
df

,name,latitude,longitude,region,pm25_one_hourly
0,west,1.35735,103.70,west,5
1,east,1.35735,103.94,east,7
2,central,1.35735,103.82,central,16
3,south,1.29587,103.82,south,22
4,north,1.41803,103.82,north,9


In [25]:
df[df['name']=='north']['pm25_one_hourly']

4    9
Name: pm25_one_hourly, dtype: int64

In [20]:
df.drop(['region'],axis=1)

,name,latitude,longitude,pm25_one_hourly
0,west,1.35735,103.70,5
1,east,1.35735,103.94,7
2,central,1.35735,103.82,16
3,south,1.29587,103.82,22
4,north,1.41803,103.82,9


### Doing analysis for UV

In [3]:
uv_data = requests.get("https://api.data.gov.sg/v1/environment/uv-index").json()
uv_data

{'items': [{'timestamp': '2021-06-26T13:00:00+08:00',
   'update_timestamp': '2021-06-26T13:08:07+08:00',
   'index': [{'value': 5, 'timestamp': '2021-06-26T13:00:00+08:00'},
    {'value': 1, 'timestamp': '2021-06-26T12:00:00+08:00'},
    {'value': 1, 'timestamp': '2021-06-26T11:00:00+08:00'},
    {'value': 2, 'timestamp': '2021-06-26T10:00:00+08:00'},
    {'value': 1, 'timestamp': '2021-06-26T09:00:00+08:00'},
    {'value': 0, 'timestamp': '2021-06-26T08:00:00+08:00'},
    {'value': 0, 'timestamp': '2021-06-26T07:00:00+08:00'}]}],
 'api_info': {'status': 'healthy'}}